<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/BDSN_SparkSQL_OLAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2023-09-13 14:11:25.977247+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


In [2]:
#import numpy as np
import pandas as pd
#import pyarrow.csv as pv
#import pyarrow.parquet as pq

#SuperStore Data
Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [3]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
#!wget -q -O Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'
!wget -q -O Orders.zip 'https://github.com/Praxis-QR/PythonForManagers/raw/main/Data/SuperStoreSalesKaggle.zip'

#Data Cleaning with Pandas

In [4]:
df = pd.read_csv('Orders.zip', delimiter = ',')
#df.head(2)
df.info()
#df.dtypes
# order_date, ship_date, sales have been incorrectly formatted as string
# year has been incorrectly formatted as integer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        51290 non-null  object 
 1   order_date      51290 non-null  object 
 2   ship_date       51290 non-null  object 
 3   ship_mode       51290 non-null  object 
 4   customer_name   51290 non-null  object 
 5   segment         51290 non-null  object 
 6   state           51290 non-null  object 
 7   country         51290 non-null  object 
 8   market          51290 non-null  object 
 9   region          51290 non-null  object 
 10  product_id      51290 non-null  object 
 11  category        51290 non-null  object 
 12  sub_category    51290 non-null  object 
 13  product_name    51290 non-null  object 
 14  sales           51290 non-null  object 
 15  quantity        51290 non-null  int64  
 16  discount        51290 non-null  float64
 17  profit          51290 non-null 

## Sales, year needs to be fixed

In [5]:
#
# df.sales has been automatically interpreted as string because of the presence of commma,
# so replace comma and convert to numeric
#

df['sales'] = df['sales'].str.replace(',', '')    # replace , with blank
df['sales'] = pd.to_numeric(df['sales'])          # convert column to numeric
df['year'] = df['year'].astype(str)          # convert column to string

In [ ]:
#df.info()

##Date looks Bad, need to be fixed

In [6]:
# what is the first and last order date ?
# we have four years of data, but date format changes ...
#df['Order Date'].describe
df['order_date'].describe
#df['ship_date'].describe

<bound method NDFrame.describe of 0          1/1/2011
1          1/1/2011
2          1/1/2011
3          1/1/2011
4          1/1/2011
            ...    
51285    31-12-2014
51286    31-12-2014
51287    31-12-2014
51288    31-12-2014
51289    31-12-2014
Name: order_date, Length: 51290, dtype: object>

In [7]:
# Some of the dates are string like dd/mm/yyyy while others are dd-mm-yyyy
# this will cause problems in converting string to real dates
# So we move all dd-mm-yyyy to dd/mm/yyyy -- could have been the other way round also
#

df['order_date'] = df['order_date'].str.replace('-','/')
df['ship_date'] = df['ship_date'].str.replace('-','/')

# convert all dates to dd/mm/yyyy format
#
df['ship_date'] = pd.to_datetime(df['ship_date'],format='%d/%m/%Y')
df['order_date'] = pd.to_datetime(df['order_date'],format='%d/%m/%Y')
#df['order_date'].describe

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        51290 non-null  object        
 1   order_date      51290 non-null  datetime64[ns]
 2   ship_date       51290 non-null  datetime64[ns]
 3   ship_mode       51290 non-null  object        
 4   customer_name   51290 non-null  object        
 5   segment         51290 non-null  object        
 6   state           51290 non-null  object        
 7   country         51290 non-null  object        
 8   market          51290 non-null  object        
 9   region          51290 non-null  object        
 10  product_id      51290 non-null  object        
 11  category        51290 non-null  object        
 12  sub_category    51290 non-null  object        
 13  product_name    51290 non-null  object        
 14  sales           51290 non-null  int64         
 15  qu

In [9]:
df.to_csv('Orders2.csv', index=False)

#Spark
https://davidadrian.cc/python-hadoop/

In [10]:
!pip3 -q install pyspark
from pyspark.sql import SparkSession
#spark9 = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok
spark9 = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [11]:
!unzip Orders.zip

Archive:  Orders.zip
  inflating: SuperStoreOrders.csv    


In [12]:
#Inferred INCORRECTLY, from CSV file
sdf9A = spark9.read.csv('SuperStoreOrders.csv', inferSchema=True,header = True, quote = '"')
#sdf9A = spark9.read.csv('SuperStoreOrders.csv', inferSchema=True,header = True)
sdf9A.dtypes

[('order_id', 'string'),
 ('order_date', 'string'),
 ('ship_date', 'string'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'string'),
 ('quantity', 'string'),
 ('discount', 'string'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [28]:
#https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
from pyspark.sql.functions import col, to_date
from pyspark.sql.functions import regexp_replace
sdf9B = sdf9A.\
        withColumn('order_date',regexp_replace('order_date','/','-')).\
        withColumn('ship_date',regexp_replace('order_date','/','-')).\
        withColumn('order_date',to_date(col('order_date'), "d-M-y")).\
        withColumn('ship_date',to_date(col('ship_date'), "d-M-y")).\
        withColumn('sales',col('sales').cast('int')).\
        withColumn('quantity',col('quantity').cast('int'))

sdf9B.dtypes

[('order_id', 'string'),
 ('order_date', 'date'),
 ('ship_date', 'date'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'int'),
 ('quantity', 'int'),
 ('discount', 'string'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [29]:
sdf9B.tail(10)

[Row(order_id='TZ-2014-8220', order_date=datetime.date(2014, 12, 31), ship_date=datetime.date(2014, 12, 31), ship_mode='Standard Class', customer_name='Christine Kargatis', segment='Home Office', state='Dar Es Salaam', country='Tanzania', market='Africa', region='Africa', product_id='OFF-HAR-10001531', category='Office Supplies', sub_category='Labels', product_name='Harbour Creations Removable Labels, Adjustable', sales=50, quantity=6, discount='0', profit=6.84, shipping_cost=2.15, order_priority='Medium', year='2014'),
 Row(order_id='CA-2014-115427', order_date=datetime.date(2014, 12, 31), ship_date=datetime.date(2014, 12, 31), ship_mode='Standard Class', customer_name='Erica Bern', segment='Corporate', state='California', country='United States', market='US', region='West', product_id='OFF-BI-10004632', category='Office Supplies', sub_category='Binders', product_name='GBC Binding covers', sales=21, quantity=2, discount='0.2', profit=6.475, shipping_cost=2.06, order_priority='Medium',

In [30]:
sdf9X.show()

+---------------+----------+----------+--------------+-----------------+-----------+-----------------+--------------+------+--------------+----------------+---------------+------------+--------------------+-----+--------+--------+---------+-------------+--------------+----+
|       order_id|order_date| ship_date|     ship_mode|    customer_name|    segment|            state|       country|market|        region|      product_id|       category|sub_category|        product_name|sales|quantity|discount|   profit|shipping_cost|order_priority|year|
+---------------+----------+----------+--------------+-----------------+-----------+-----------------+--------------+------+--------------+----------------+---------------+------------+--------------------+-----+--------+--------+---------+-------------+--------------+----+
|   AG-2011-2040|2011-01-01|2011-01-01|Standard Class|  Toby Braunhardt|   Consumer|      Constantine|       Algeria|Africa|        Africa|OFF-TEN-10000025|Office Supplies|   

In [ ]:
sdf9A.select('sales','quantity').show(20)

+-----+--------+
|sales|quantity|
+-----+--------+
|  408|       2|
|  120|       3|
|   66|       4|
|   45|       3|
|  114|       5|
|   55|       2|
|  314|       1|
|  276|       1|
|  912|       4|
|  667|       4|
|  338|       3|
|  211|       1|
|  854|       7|
|  193|       1|
|  159|       2|
|  195|       4|
|  123|       2|
|   69|       2|
|   69|       2|
|  135|       2|
+-----+--------+
only showing top 20 rows



In [ ]:
sdf9B = spark9.read.csv('Orders2.csv', inferSchema=True,header = True)
sdf9B.dtypes

[('order_id', 'string'),
 ('order_date', 'date'),
 ('ship_date', 'date'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'string'),
 ('quantity', 'string'),
 ('discount', 'string'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [ ]:
#Inferred CORRECTLY, from parquet file
sdf9B = spark9.read.parquet('Orders2.parquet', inferSchema=True,header = True, quote = '"')
sdf9B.dtypes

[('order_id', 'string'),
 ('order_date', 'timestamp_ntz'),
 ('ship_date', 'timestamp_ntz'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'bigint'),
 ('quantity', 'bigint'),
 ('discount', 'double'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [ ]:
# Reduced Data
#selCols = ['Order ID','Order Date','Segment','Region','Category','Sales','Quantity','Profit']
selCols = ['year','segment','region','category','sales','quantity','profit']
#df9C = spark9.read.parquet('Orders2.parquet',inferSchema=True,header = True, quote = '"').select('Order ID','Order Date','Segment','Region','Category','Sales','Quantity','Profit')
sdf9C = spark9.read.parquet('Orders2.parquet',inferSchema=True,header = True, quote = '"').select(selCols)
#df9.dtypes
sdf9C.dtypes

[('year', 'string'),
 ('segment', 'string'),
 ('region', 'string'),
 ('category', 'string'),
 ('sales', 'bigint'),
 ('quantity', 'bigint'),
 ('profit', 'double')]

In [ ]:
# remove , and " from data
#import pandas as pd
#p_df = pd.read_csv('SSdatafile.csv')
#p_df = pd.read_csv('datafile.csv')
#p_df2 = p_df.replace(',', ' ', regex=True)
#p_df3 = p_df2.replace('"', '', regex=True)
#p_df3.to_csv('SSdatafile2.csv',index=False,sep=',')
#!head SSdatafile2.csv

##Spark SQL

In [ ]:
spark9.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [ ]:
#air_quality_sdf.createOrReplaceTempView("air_quality_sdf")
sdf9C.createOrReplaceTempView("orders_temp")

In [ ]:
spark9.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |orders_temp|       true|
+---------+-----------+-----------+



In [ ]:
spark9.sql("desc orders_temp").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    year|   string|   null|
| segment|   string|   null|
|  region|   string|   null|
|category|   string|   null|
|   sales|   bigint|   null|
|quantity|   bigint|   null|
|  profit|   double|   null|
+--------+---------+-------+



In [ ]:
spark9.sql('select * from Orders_Temp limit 5').show()

+----+-----------+-------+---------------+-----+--------+-------+
|year|    segment| region|       category|sales|quantity| profit|
+----+-----------+-------+---------------+-----+--------+-------+
|2011|   Consumer| Africa|Office Supplies|  408|       2| 106.14|
|2011|   Consumer|Oceania|Office Supplies|  120|       3| 36.036|
|2011|   Consumer|   EMEA|Office Supplies|   66|       4|  29.64|
|2011|Home Office|  North|Office Supplies|   45|       3|-26.055|
|2011|   Consumer|Oceania|      Furniture|  114|       5|  37.77|
+----+-----------+-------+---------------+-----+--------+-------+



In [ ]:
sql_drop_table = """
drop table if exists praxisdb.sh_orders
"""

sql_drop_database = """
drop database if exists praxisdb cascade
"""

sql_create_database = """
create database if not exists praxisdb
location '/content/praxisdb/'
"""

sql_create_table = """
create table if not exists praxisdb.sh_orders
using parquet
as select  * from Orders_Temp
"""

#as select to_timestamp(date) as date_parsed, *

print("dropping database...")
result_drop_db = spark9.sql(sql_drop_database)

print("creating database...")
result_create_db = spark9.sql(sql_create_database)

print("dropping table...")
result_droptable = spark9.sql(sql_drop_table)

print("creating table...")
result_create_table = spark9.sql(sql_create_table)

dropping database...
creating database...
dropping table...
creating table...


In [ ]:
spark9.sql("use praxisdb")
spark9.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
| praxisdb|  sh_orders|      false|
|         |orders_temp|       true|
+---------+-----------+-----------+



In [ ]:
spark9.sql("select * from praxisdb.sh_orders").show(5)
#.select("date_parsed", "O_3").show(5)

+----+-----------+-------+---------------+-----+--------+-------+
|year|    segment| region|       category|sales|quantity| profit|
+----+-----------+-------+---------------+-----+--------+-------+
|2011|   Consumer| Africa|Office Supplies|  408|       2| 106.14|
|2011|   Consumer|Oceania|Office Supplies|  120|       3| 36.036|
|2011|   Consumer|   EMEA|Office Supplies|   66|       4|  29.64|
|2011|Home Office|  North|Office Supplies|   45|       3|-26.055|
|2011|   Consumer|Oceania|      Furniture|  114|       5|  37.77|
+----+-----------+-------+---------------+-----+--------+-------+
only showing top 5 rows



In [ ]:
spark9.sql("select distinct(region) from praxisdb.sh_orders").show()

+--------------+
|        region|
+--------------+
|        Africa|
|    North Asia|
|  Central Asia|
|         South|
|     Caribbean|
|       Central|
|          East|
|       Oceania|
|        Canada|
|          West|
|          EMEA|
|Southeast Asia|
|         North|
+--------------+



##CrossTab

###Inner Query

In [ ]:
sums_df = spark9.sql("""
    SELECT
        Segment,
        SUM(IF(Region='Africa',Sales,0)) As Africa,
        SUM(IF(Region='North Asia',Sales,0)) As North_Asia,
        SUM(IF(Region='Central Asia',Sales,0)) As Central_Asia,
        SUM(IF(Region='Caribbean',Sales,0)) As Caribbean,
        SUM(IF(Region='Oceania',Sales,0)) As Oceania,
        SUM(IF(Region='Canada',Sales,0)) As Canada,
        SUM(IF(Region='EMEA',Sales,0)) As EMEA,
        SUM(IF(Region='South East Asia',Sales,0)) As SE_Asia,
        SUM(IF(Region='Central',Sales,0)) As Central,
        SUM(IF(Region='East',Sales,0)) As East,
        SUM(IF(Region='West',Sales,0)) As West,
        SUM(IF(Region='North',Sales,0)) As North,
        SUM(IF(Region='South',Sales,0)) As South
    FROM praxisdb.sh_orders
        GROUP BY Segment WITH ROLLUP
""")
sums_df.show()

+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+-------+
|    Segment|Africa|North_Asia|Central_Asia|Caribbean|Oceania|Canada|  EMEA|SE_Asia|Central|  East|  West|  North|  South|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+-------+
|Home Office|155077|    175177|      148235|    57395| 197812| 11896|148862|      0| 491961|127465|136736| 209996| 280016|
|  Corporate|204948|    265762|      235552|   104535| 322831| 19319|250559|      0| 850405|200422|225870| 394220| 496029|
|       null|783776|    848349|      752839|   324281|1100207| 66932|806184|      0|2822399|678834|725514|1248192|1600960|
|   Consumer|423751|    407410|      369052|   162351| 579564| 35717|406763|      0|1480033|350947|362908| 643976| 824915|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+-------+



###Outer Query

In [ ]:
#sums_df.registerTempTable("sums")
sums_df.createOrReplaceTempView("sums_T")
#CrossTab = sqlContext.sql("""
CrossTab = spark9.sql("""
 SELECT
  IFNULL(Segment,'#Totals') Dims,
  Africa,North_Asia,Central_Asia,Caribbean,
  Oceania,Canada,EMEA,SE_Asia,
  Central,East,West,South,
  Africa +North_Asia + Central_Asia+ Caribbean + Oceania + Canada+EMEA + SE_Asia + Central + East + West + South Totals
FROM sums_t
order by Dims desc
""")
CrossTab.show()

+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|       Dims|Africa|North_Asia|Central_Asia|Caribbean|Oceania|Canada|  EMEA|SE_Asia|Central|  East|  West|  South|  Totals|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|Home Office|155077|    175177|      148235|    57395| 197812| 11896|148862|      0| 491961|127465|136736| 280016| 1930632|
|  Corporate|204948|    265762|      235552|   104535| 322831| 19319|250559|      0| 850405|200422|225870| 496029| 3176232|
|   Consumer|423751|    407410|      369052|   162351| 579564| 35717|406763|      0|1480033|350947|362908| 824915| 5403411|
|    #Totals|783776|    848349|      752839|   324281|1100207| 66932|806184|      0|2822399|678834|725514|1600960|10510275|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+



###Using Subquery

In [ ]:
CrossTab2 = spark9.sql("""
 SELECT
  IFNULL(Segment,'#Totals') Dims,
  Africa,North_Asia,Central_Asia,Caribbean,
  Oceania,Canada,EMEA,SE_Asia,
  Central,East,West,South,
  Africa +North_Asia + Central_Asia+ Caribbean + Oceania + Canada+EMEA + SE_Asia + Central + East + West + South Totals
FROM (
SELECT
        Segment,
        SUM(IF(Region='Africa',Sales,0)) As Africa,
        SUM(IF(Region='North Asia',Sales,0)) As North_Asia,
        SUM(IF(Region='Central Asia',Sales,0)) As Central_Asia,
        SUM(IF(Region='Caribbean',Sales,0)) As Caribbean,
        SUM(IF(Region='Oceania',Sales,0)) As Oceania,
        SUM(IF(Region='Canada',Sales,0)) As Canada,
        SUM(IF(Region='EMEA',Sales,0)) As EMEA,
        SUM(IF(Region='South East Asia',Sales,0)) As SE_Asia,
        SUM(IF(Region='Central',Sales,0)) As Central,
        SUM(IF(Region='East',Sales,0)) As East,
        SUM(IF(Region='West',Sales,0)) As West,
        SUM(IF(Region='North',Sales,0)) As North,
        SUM(IF(Region='South',Sales,0)) As South
    FROM praxisdb.sh_orders
        GROUP BY Segment WITH ROLLUP
    )
as sums
order by Dims desc
""")
CrossTab2.show()

+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|       Dims|Africa|North_Asia|Central_Asia|Caribbean|Oceania|Canada|  EMEA|SE_Asia|Central|  East|  West|  South|  Totals|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|Home Office|155077|    175177|      148235|    57395| 197812| 11896|148862|      0| 491961|127465|136736| 280016| 1930632|
|  Corporate|204948|    265762|      235552|   104535| 322831| 19319|250559|      0| 850405|200422|225870| 496029| 3176232|
|   Consumer|423751|    407410|      369052|   162351| 579564| 35717|406763|      0|1480033|350947|362908| 824915| 5403411|
|    #Totals|783776|    848349|      752839|   324281|1100207| 66932|806184|      0|2822399|678834|725514|1600960|10510275|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+



In [ ]:
spark9.sql("select distinct(year) from praxisdb.sh_orders").show()

+----+
|year|
+----+
|2012|
|2014|
|2013|
|2011|
+----+



In [ ]:
CrossTab2 = spark9.sql("""
 SELECT
  IFNULL(Category,'#Totals') Dims,
  Y2011,Y2012,Y2013,Y2014,Y2011+Y2012+Y2013+Y2014 Totals
FROM (
SELECT
        Category,
        SUM(IF(Year='2011',Sales,0)) As Y2011,
        SUM(IF(Year='2012',Sales,0)) As Y2012,
        SUM(IF(Year='2013',Sales,0)) As Y2013,
        SUM(IF(Year='2014',Sales,0)) As Y2014
    FROM praxisdb.sh_orders
        GROUP BY Category WITH ROLLUP
    )
as sums
order by Dims desc
""")
CrossTab2.show()

+---------------+-------+-------+-------+-------+--------+
|           Dims|  Y2011|  Y2012|  Y2013|  Y2014|  Totals|
+---------------+-------+-------+-------+-------+--------+
|     Technology| 827698|1023467|1277328|1616198| 4744691|
|Office Supplies| 675642| 795113|1010793|1305782| 3787330|
|      Furniture| 756171| 858913|1117739|1378061| 4110884|
|        #Totals|2259511|2677493|3405860|4300041|12642905|
+---------------+-------+-------+-------+-------+--------+



#Parquet to Pandas again

In [ ]:
!ls /content/praxisdb/sh_orders

part-00000-2a504e60-1ed6-4c94-8178-34ffdb8e914a-c000.snappy.parquet  _SUCCESS


In [ ]:
df7 = pd.read_parquet('/content/praxisdb/sh_orders/part-00000-2a504e60-1ed6-4c94-8178-34ffdb8e914a-c000.snappy.parquet')
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      51290 non-null  object 
 1   segment   51290 non-null  object 
 2   region    51290 non-null  object 
 3   category  51290 non-null  object 
 4   sales     51290 non-null  int64  
 5   quantity  51290 non-null  int64  
 6   profit    51290 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 2.7+ MB


#Second Spark

In [ ]:
spark39 = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()
target_parquet = 'Orders2.parquet'
sdf39B = spark39.read.parquet(target_parquet, inferSchema=True,header = True, quote = '"')
sdf39B.dtypes

[('order_id', 'string'),
 ('order_date', 'timestamp_ntz'),
 ('ship_date', 'timestamp_ntz'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'bigint'),
 ('quantity', 'bigint'),
 ('discount', 'double'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [ ]:
sdf39B.repartition(2).write.mode('overwrite').parquet('wpart/Orders239')

In [ ]:
spark39.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
| praxisdb|
+---------+



#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)